In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class OutlierClipper(BaseEstimator, TransformerMixin):
    """Clip features by lower/upper quantiles per-column (works on numpy arrays)."""
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        self.lower_bounds_ = None
        self.upper_bounds_ = None
    
    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        # compute per-column quantiles
        self.lower_bounds_ = np.quantile(X, self.lower_quantile, axis=0)
        self.upper_bounds_ = np.quantile(X, self.upper_quantile, axis=0)
        return self
    
    def transform(self, X):
        X = np.asarray(X, dtype=float).copy()
        X = np.clip(X, self.lower_bounds_, self.upper_bounds_)
        return X


In [3]:
import joblib
import pandas as pd

# Load pipeline and label encoder
best_model = joblib.load("models/best_model_RandomForest.pkl")  # or XGBoost file
le = joblib.load("models/label_encoder.pkl")

# Column names (must match training)
numeric_features = ['N','P','K','temperature','humidity','ph','rainfall']


In [8]:
sample = {'N':90, 'P':42, 'K':43, 'temperature':40, 'humidity':0, 'ph':2.5, 'rainfall':100}
row = pd.DataFrame([sample], columns=numeric_features)
pred_enc = best_model.predict(row)[0]
pred_crop = le.inverse_transform([pred_enc])[0]
print("Recommended Crop:", pred_crop)


Recommended Crop: kidneybeans


In [9]:
from ipywidgets import widgets, VBox
from IPython.display import display

inputs = {feat: widgets.FloatText(value=50, description=feat) for feat in numeric_features}
button = widgets.Button(description="Predict Crop")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        sample = pd.DataFrame([{k: v.value for k,v in inputs.items()}])
        pred_enc = best_model.predict(sample)[0]
        pred_crop = le.inverse_transform([pred_enc])[0]
        print("✅ Recommended Crop:", pred_crop)

        # Top 3 probabilities
        if hasattr(best_model.named_steps['model'], "predict_proba"):
            probs = best_model.predict_proba(sample)[0]
            top_idx = probs.argsort()[::-1][:3]
            print("Top 3 crop predictions:")
            for i in top_idx:
                print(f"{le.inverse_transform([i])[0]} : {probs[i]:.2f}")

button.on_click(on_button_click)
display(VBox(list(inputs.values()) + [button, output]))
